In [18]:
from Config import *
from HotelModel import *
import fileUtil
import re
import os

In [20]:
basic_dir = '../dataset'
# file_name = 'generate_select_distinct_name_zh_cn_temp.csv'
stop_file_name = 'stopwords.txt'

file_name = '../dataset/remove_stopwords.csv'
weight_file = 'hotels_model.h5'
# 根据前六个字预测第七个字
max_len = 6
batch_size = 512
learning_rate = 0.1

In [21]:
if os.path.exists(os.path.join(basic_dir,stop_file_name)):
    stopWords = fileUtil.read_data_as_array(stop_file_name)
else:
    quit()
stopword_list = set(stopWords)

documents = []
with open(os.path.join(basic_dir,file_name), 'r', encoding='utf-8') as f:
    while True:
        line = f.readline()
        if len(line.strip()) == 0:
            print('段落处理完毕')
        if not line:
            break
        for char in stopword_list:
            line = line.replace(char, "")
        line = line.replace('。。', "END")
        line = line.replace('。', "END")
        documents.append(line)

段落处理完毕
段落处理完毕


In [22]:
print(documents[:30])

['上海时生隅园林文化酒店2017年的秋末，时生隅初见END彼时的心愿，想要在快节奏的都市，寻处讲诉时光和生命的角落END错，不闻浮沉END这便是时生隅的由来END，我们秉承初心，在闹中取静的川沙地铁站商圈，觅方天地—充满江南气息的园林，叶园END长廊湖景，古雅庭院END淡雅而大气，是快节奏与民俗文化的交融，幽静而明快，是远道而来或憩而居的别样天地END时生隅的新老朋友，我们在叶园，期待与您共襄景，时光言欢END\n', '云和夜泊酒店上海浦东国际机场店，位于浦东新区祝桥秋亭路，毗邻上海浦东国际机场，车程20分钟，距上海迪士尼18分钟车程，距上海野生动物园21分钟车程END酒店地理位置优越，并配备大中豪华轿车接送客户END酒店拥有房间98间，大床30间，行政大床15间，标间30间，亲子房23间END\n', '维纳酒店上海浦东机场店位于浦东新区川南奉公路，近晨阳路，与迪士尼直线距离约8公里，可便捷到达地铁2号线凌空路站，交通便利END酒店周围生活设施齐全，旅游资源众多，有上海新国际博览中心迪士尼上海野生动物园农艺大观园三甲港滨海旅游区END维纳酒店上海浦东机场店是维纳旗的连锁酒店，装修豪华舒适，整体风格高贵典雅END客房宽敞明亮温馨时尚，房内布置精美，处处体现人性化的理念END酒店还有宽敞停车场休闲茶宽敞会议室，同时还为您提供精品早餐浦东机场接机需预约服务，是商务休闲会务的理想酒店END\n', '上海万信R酒店位于浦东新区崮山路，地处内环线陆家嘴金融区内，近地铁6号线北洋泾路站，地铁9号线芳甸路站，配备专属停车场，自驾至新国际博览中心约10分钟，至外滩世博园区南京路步行街上海自由贸易区上海迪斯尼乐园约20分钟END酒店拥有百余间崭新客房，精选配套，舒适与轻奢相拥酒店圣罗拉餐厅环境舒适安逸，主西餐辅以粤菜川菜沪菜，在此享受由港粤名厨精心烹饪的佳肴，舌尖不由自主的跳了芭蕾END早餐的菜品琳琅满目可达40种，为保证口味，餐品均选优质食材天加工END酒店豪华宴会厅提供站式婚礼服务与专业的会务服务END200㎡万信厅可支持约120人课桌式会议，100㎡丽都厅适合60人以的各类会议需求END万信R酒店设计，突破了万信酒店贯理念，时尚年轻，秉承R文化，华丽，贴心，轻松，享受END\n', '酒店位于浦东新区沪南公路3655弄2号，素有上海之称的周浦镇，沪南公路横桥路路口，周边有

In [14]:
puncs = [']', '[', '（', '）', '{', '}', '：', '《', '》']
def preprocess_file():
    # 语料文本内容
    files_content = ''
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            # 每行的末尾加上"]"符号代表一首诗结束
            for char in puncs:
                line = line.replace(char, "")
            files_content += line.strip() + "]"
    words = sorted(list(files_content))
    words.remove(']')
    counted_words = {}
    for word in words:
        if word in counted_words:
            counted_words[word] += 1
        else:
            counted_words[word] = 1

    # 去掉低频的字
    erase = []
    for key in counted_words:
        if counted_words[key] <= 2:
            erase.append(key)
    for key in erase:
        del counted_words[key]
    del counted_words[']']
    wordPairs = sorted(counted_words.items(), key=lambda x: -x[1])
    
    words, _ = zip(*wordPairs)
    print(_)
    # word到id的映射
    word2num = dict((c, i + 1) for i, c in enumerate(words))
    num2word = dict((i, c) for i, c in enumerate(words))
    word2numF = lambda x: word2num.get(x, 0)
    return word2numF, num2word, words, files_content

In [15]:
word2numF, num2word, words, files_content = preprocess_file()

(117, 50, 46, 46, 46, 44, 42, 31, 24, 21, 21, 20, 19, 18, 18, 17, 16, 16, 16, 16, 15, 15, 15, 14, 14, 14, 14, 13, 12, 12, 12, 12, 11, 11, 11, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3)


In [ ]:
model = None
do_train = True
self.loaded_model = False
self.config = config

if os.path.exists(weight_file):
    model = build_model()
    model.load_weights(tf.train.latest_checkpoint(self.config.weight_file))
    # load_model(self.config.weight_file)
    model.summary()
else:
    train()
do_train = False
loaded_model = True

In [ ]:
def build_model(self):
    '''建立模型'''
    input_tensor = tf.keras.layers.Input(shape=(config.max_len,))
    embedd = tf.keras.layers.Embedding(len(num2word)+1, 300, input_length=config.max_len)(input_tensor)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embedd)
    dropout = tf.keras.layers.Dropout(0.6)(lstm)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embedd)
    dropout = tf.keras.layers.Dropout(0.6)(lstm)
    flatten = tf.keras.layers.Flatten()(lstm)
    dense = tf.keras.layers.Dense(len(words), activation='softmax')(flatten)
    model = tf.keras.models.Model(inputs=input_tensor, outputs=dense)
    optimizer = tf.keras.optimizers.Adam(lr=self.config.learning_rate)
    self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
def sample(self, preds, temperature=1.0):
    '''
    当temperature=1.0时，模型输出正常
    当temperature=0.5时，模型输出比较open
    当temperature=1.5时，模型输出比较保守
    在训练的过程中可以看到temperature不同，结果也不同
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def train(self):
    number_of_epoch = 10
    if not self.model:
        self.build_model()
    self.model.summary()
    self.model.fit_generator(
        generator=self.data_generator(),
        verbose=True,
        steps_per_epoch=self.config.batch_size,
        epochs=number_of_epoch,
        callbacks=[
            keras.callbacks.ModelCheckpoint(self.config.weight_file, save_weights_only=False),
            keras.callbacks.LambdaCallback(on_epoch_end=self.generate_sample_result)
        ])

In [ ]:
def generate_sample_result(self, epoch, logs):
    print("\n==================Epoch {}=====================".format(epoch))
    for diversity in [0.5, 1.0, 1.5]:
        print("------------Diversity {}--------------".format(diversity))
        start_index = random.randint(0, len(self.files_content) - self.config.max_len - 1)
        generated = ''
        sentence = self.files_content[start_index: start_index + self.config.max_len]
        generated += sentence
        for i in range(20):
            x_pred = np.zeros((1, self.config.max_len))
            for t, char in enumerate(sentence[-6:]):
                x_pred[0, t] = self.word2numF(char)

            preds = self.model.predict(x_pred, verbose=0)[0]
            next_index = self.sample(preds, diversity)
            next_char = self.num2word[next_index]
            generated += next_char
            sentence = sentence + next_char
        print(sentence)

In [ ]:
def predict(self, text):
    if not self.loaded_model:
            return
    with open(self.config.file_name, 'r', encoding='utf-8') as f:
        file_list = f.readlines()
    random_line = random.choice(file_list)
    # 如果给的text不到四个字，则随机补全
    if not text or len(text) != 4:
        for _ in range(4 - len(text)):
            random_str_index = random.randrange(0, len(self.words))
            text += self.num2word.get(random_str_index) if self.num2word.get(random_str_index) not in [',', '。','，'] else self.num2word.get(random_str_index + 1)
    seed = random_line[-(self.config.max_len):-1]
    res = ''
    seed = 'c' + seed
    for c in text:
        seed = seed[1:] + c
        for j in range(5):
            x_pred = np.zeros((1, self.config.max_len))
            for t, char in enumerate(seed):
                x_pred[0, t] = self.word2numF(char)
            preds = self.model.predict(x_pred, verbose=0)[0]
            next_index = self.sample(preds, 1.0)
            next_char = self.num2word[next_index]
            seed = seed[1:] + next_char
        res += seed
    return res

In [ ]:
def data_generator(self):
    i = 0
    while 1:
        x = self.files_content[i: i + self.config.max_len]
        y = self.files_content[i + self.config.max_len]
        puncs = [']', '[', '（', '）', '{', '}', '：', '《', '》', ':']
        if len([i for i in puncs if i in x]) != 0:
            i += 1
            continue
        if len([i for i in puncs if i in y]) != 0:
            i += 1
            continue
        y_vec = np.zeros(
            shape=(1, len(self.words)),
            dtype=np.bool
        )
        y_vec[0, self.word2numF(y)] = 1.0
        x_vec = np.zeros(
            shape=(1, self.config.max_len),
            dtype=np.int32
        )
        for t, char in enumerate(x):
            x_vec[0, t] = self.word2numF(char)
        yield x_vec, y_vec
        i += 1